### Imports and data cleaning

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy import signal
from scipy.signal import savgol_filter
from sklearn.metrics import mean_squared_error, r2_score
from IPython import display
import time
import IPython
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Text, Button
from IPython.display import display
import matplotlib.pyplot as plt

In [ ]:
dataframe3 = pd.read_excel('COVID-19 US state policy database 6_10_2021.xlsx')
df2 = pd.read_csv('time_series_covid_19_confirmed_US.csv')

In [ ]:
states = (df2['Province_State'].unique())
row = []
popValues = []
for state in states:
    r = df2.loc[df2["Province_State"] == state, "5/29/21"]
    sum_row = r.sum(axis=0)
    row.append(sum_row)
df = pd.DataFrame(row) 
df.columns = ["Total Covid Cases"]
for index, state in enumerate(states):
    df.rename(index={index: state}, inplace=True)
df = df.drop(["American Samoa", "Diamond Princess", "Grand Princess", "Northern Mariana Islands", "Guam", "Puerto Rico", "Virgin Islands" ], axis=0)
r = dataframe3.loc[:,"POP18"]
for index, val in enumerate(r):
    if index > 3:
        popValues.append(val)
df["Population 2018"] = popValues

sns.lmplot(x ="Population 2018", y ="Total Covid Cases", data = df, order = 2, ci = None)


### Dominiques Edits - Added Sliders Linear Regression

In [ ]:
X = np.array(df['Population 2018']).reshape(-1, 1)
y = np.array(df['Total Covid Cases']).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.3)

""""
alpha = 10 ** -16
iterations = 1000
num_image_updates = 20
parameters = linear_regression(X_test, y_test, alpha, iterations, num_image_updates)
print(parameters)


#doesn't the lin regression function above already calculate and plot the predictions (line)?
y_pred = np.empty([len(X_test), 1])
for i in range(len(X_test)):
    y_pred[i]  = parameters[0]*X_test[i] + parameters[1]

plt.scatter(X_test, y_test, color ='b')
plt.plot(X_test, y_pred, color ='k')
plt.show()
"""

In [ ]:
#Linear Regression function attempting to store y_predicted values so that they can be displayed later? Still having issues with
# displaying updates to the graph
def linear_regression(X_test, y_test, alpha, iterations, num_image_updates):
    w = 0
    b = 0
    y_preds = []
    step_size = iterations/num_image_updates

    plt.ion()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.scatter(X_test, y_test, color ='b')
    init = range(len(X_test))
    line, = ax.plot(X_test, init, 'k')

    for i in range(iterations):
        dJ_dw = 0
        dJ_db = 0
        for j in range(len(X_test)):
            dJ_dw = dJ_dw + (1/len(X_test))*(w*X_test[j] + b - y_test[j])*X_test[j]
            dJ_db = dJ_db + (1/len(X_test))*(w*X_test[j] + b - y_test[j])
        w = w - alpha*dJ_dw
        b = b - alpha*dJ_db

        y_pred = np.empty([len(X_test), 1])
        for k in range(len(X_test)):
            y_pred[k] = w*X_test[k] + b

        y_preds.append(y_pred)

        if i % step_size == 0:
            line.set_ydata(y_pred)
            fig.canvas.draw()
            fig.canvas.flush_events()
            """
            plt.clf()
            plt.scatter(X_test, y_test, color ='b')
            plt.plot(X_test, y_pred, color ='k')
            IPython.display.display(plt.gcf())
            IPython.display.clear_output(wait=True)
            time.sleep(1)
            """
    
    parameters = (w, b, y_preds)
    return parameters
    

In [ ]:
#default values
alpha = 10 ** -16
iterations = 1000
num_image_updates = 20
y_preds = []

#Slider functions - alpha having issues bc it is so small so might need text input?
def alpha_update(alpha_in):
    alpha = alpha_in

def iterations_update(iterations_in):
    iteraions = iterations_in

def num_image_updates_update(num_image_updates_in):
    num_image_updates = num_image_updates_in

def run_only_lin_reg(button):
    #interact(linear_regression, X_test = X_test, y_test = y_test, alpha = alpha, iterations = iterations, num_image_updates = num_image_updates)
    (w, b, y_preds_return) =linear_regression(X_test, y_test, alpha, iterations, num_image_updates)
    for i in range(len(y_preds_return)):
        y_preds.append(y_preds_return[i])

In [ ]:
#interact(alpha_update, alpha_in=alpha)
interact(iterations_update, iterations_in=iterations)
interact(num_image_updates_update, num_image_updates_in=num_image_updates)

button = widgets.Button(description='Enter')
button.on_click(run_only_lin_reg)
output = widgets.Output()

#display(button, output)
display(button, output)

In [ ]:
#Plotting linear regression - can't seem to get this to work in the interactive section
plt.ion()

fig = plt.figure()
ax = fig.add_subplot(111)
line, = ax.plot(X_test, y_preds[0], 'b')
step_size = iterations/num_image_updates

for i in range(len(y_preds)):
    if i % step_size == 0:
        plt.clf()
        plt.scatter(X_test, y_test, color ='b')
        plt.plot(X_test, y_preds[i], color ='k')
        IPython.display.display(plt.gcf())
        IPython.display.clear_output(wait=True)

### Sola's Edits

In [ ]:
def f(x):
    return x

def g(w, b):
    if(w != '' and b != ''):
        return int(w) + int(b)

In [ ]:
interact(f, x=10)

In [ ]:
interact(g, w = '', b = '')

In [ ]:
#version of linear regression with w, b text box functionality
def linear_regression(X_test, y_test, alpha, iterations, num_image_updates, has_w_b, w, b):
    
    if has_w_b == False:
        
        step_size = iterations/num_image_updates
    
        for i in range(iterations):
            dJ_dw = 0
            dJ_db = 0
            for j in range(len(X_test)):
                dJ_dw = dJ_dw + (1/len(X_test))*(w*X_test[j] + b - y_test[j])*X_test[j]
                dJ_db = dJ_db + (1/len(X_test))*(w*X_test[j] + b - y_test[j])
            w = w - alpha*dJ_dw
            b = b - alpha*dJ_db

            y_pred = np.empty([len(X_test), 1])
            for k in range(len(X_test)):
                y_pred[k]  = w*X_test[k] + b

            if i % step_size == 0:
                plt.clf()
                plt.scatter(X_test, y_test, color ='b')
                plt.plot(X_test, y_pred, color ='k')
                display.display(plt.gcf())
                display.clear_output(wait=True)
                time.sleep(1)
    else:
        y_pred = np.empty([len(X_test), 1])
        for k in range(len(X_test)):
            y_pred[k]  = w*X_test[k] + b

        
        plt.clf()
        plt.scatter(X_test, y_test, color ='b')
        plt.plot(X_test, y_pred, color ='k')
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(1)
    
    parameters = (w, b)
    return parameters

In [ ]:
def run_lin_reg(has_w_b, w, b):
    
    X = np.array(df['Population 2018']).reshape(-1, 1)
    y = np.array(df['Total Covid Cases']).reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.3)

    #make it so this info is obtained from sliders instead
    alpha = 10 ** -16
    iterations = 1000
    num_image_updates = 20
    
    #replace has_w_b text with the result from the button or textbox!!!
    if(w != '' and b != '' and has_w_b == "True"):
        w = float(w)
        b = float(b)

        X = np.array(df['Population 2018']).reshape(-1, 1)
        y = np.array(df['Total Covid Cases']).reshape(-1, 1)

        
        parameters = linear_regression(X_test, y_test, alpha, iterations, num_image_updates, True, w, b)

        print(parameters)

        y_pred = np.empty([len(X_test), 1])
        for i in range(len(X_test)):
            y_pred[i]  = parameters[0]*X_test[i] + parameters[1]

        plt.scatter(X_test, y_test, color ='b')
        plt.plot(X_test, y_pred, color ='k')
        plt.show()

    elif has_w_b == "False":
        parameters = linear_regression(X_test, y_test, alpha, iterations, num_image_updates, False, 0, 0)

        print(parameters)

        y_pred = np.empty([len(X_test), 1])
        for i in range(len(X_test)):
            y_pred[i]  = parameters[0]*X_test[i] + parameters[1]

        plt.scatter(X_test, y_test, color ='b')
        plt.plot(X_test, y_pred, color ='k')
        plt.show()
    return

In [ ]:
def interact_true(button):
    #maybe make it so the parameters do not show?
    interact(run_lin_reg, has_w_b = 'True', w = '', b = '')

In [ ]:
def interact_false(button):
    #interactive so it does not show w and b, why is it not running lin_reg correctly
    interact(run_lin_reg, has_w_b = 'False', w = '', b = '')

In [ ]:
run_button_with_wb = widgets.Button(description='Run WITH w and b')
run_button_with_wb.on_click(interact_true)
run_button_without_wb = widgets.Button(description='Run WITHOUT w and b')
run_button_without_wb.on_click(interact_false)
output = widgets.Output()

display(run_button_with_wb, run_button_without_wb, output)

### Sheel's Edits

In [ ]:
#version of linear regression with w, b text box functionality

def linear_regression_EDITED(X_test, y_test, alpha, iterations, num_image_updates, has_w_b, w, b):

   
    #has_w_b = last_three_params[0]
    #w = last_three_params[1]
    #b = last_three_params[2]
    
    if has_w_b == False:
        w = 0
        b = 0
        
        step_size = iterations/num_image_updates
    
        for i in range(iterations):
            dJ_dw = 0
            dJ_db = 0
            for j in range(len(X_test)):
                dJ_dw = dJ_dw + (1/len(X_test))*(w*X_test[j] + b - y_test[j])*X_test[j]
                dJ_db = dJ_db + (1/len(X_test))*(w*X_test[j] + b - y_test[j])
            w = w - alpha*dJ_dw
            b = b - alpha*dJ_db

            y_pred = np.empty([len(X_test), 1])
            for k in range(len(X_test)):
                y_pred[k]  = w*X_test[k] + b

            if i % step_size == 0:
                plt.clf()
                plt.scatter(X_test, y_test, color ='b')
                plt.plot(X_test, y_pred, color ='k')
                IPython.display.display(plt.gcf())
                IPython.display.clear_output(wait=True)
                time.sleep(1)
    else:
        y_pred = np.empty([len(X_test), 1])
        for k in range(len(X_test)):
            y_pred[k]  = w*X_test[k] + b

        
        plt.clf()
        plt.scatter(X_test, y_test, color ='b')
        plt.plot(X_test, y_pred, color ='k')
        IPython.display.display(plt.gcf())
        IPython.display.clear_output(wait=True)
        time.sleep(1)
    
    parameters = (w, b)
    return parameters

In [ ]:
#Sheel's edits: two buttons to run the linear regression with and without wb
#issues: doesn't clear the image; instead, it displays one after another and then replaces all with final output

#also, each run doesn't remove the prevous run's graph!! 

#NOTE: when running with given w and b that are very far from optimized w and b, the scale of the
#plot changes. The scatterplot isn't changing, the best fit line is just so far off that the scale 
#was altered drastically to show both the points and line. 




def run_lin_reg_with_wb(button):
    w = w_input.value
    b = b_input.value
    

    w = float(w)
    b = float(b)

    print(w)
    print(b)

    parameters = (w, b)
    
    X = np.array(df['Population 2018']).reshape(-1, 1)
    y = np.array(df['Total Covid Cases']).reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.3)

    alpha = 10 ** -16
    iterations = 1000
    num_image_updates = 20


    # parameters = linear_regression_EDITED(X_test, y_test, alpha, iterations, num_image_updates, True, w, b)

    # print(parameters)

    y_pred = np.empty([len(X_test), 1])
    for i in range(len(X_test)):
        y_pred[i] = parameters[0]*X_test[i] + parameters[1]
    

    plt.scatter(X_test, y_test, color ='b')
    plt.plot(X_test, y_pred, color ='k')
    plt.show()


def run_lin_reg_without_wb(button):
    #set w and b to 0, not important because not used
    w = 0
    b = 0

    parameters = (0, 0)

    X = np.array(df['Population 2018']).reshape(-1, 1)
    y = np.array(df['Total Covid Cases']).reshape(-1, 1)

    #with this button clicked, user did NOT provided us with w and b
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, train_size=0.3)

    alpha = 10 ** -16
    iterations = 1000
    num_image_updates = 20

    #with this button clicked, user did NOT provided us with w and b
    parameters = linear_regression_EDITED(X_test, y_test, alpha, iterations, num_image_updates, False, w, b)

    print(parameters)

    y_pred = np.empty([len(X_test), 1])
    for i in range(len(X_test)):
        y_pred[i] = parameters[0]*X_test[i] + parameters[1]

    plt.scatter(X_test, y_test, color ='b')
    plt.plot(X_test, y_pred, color ='k')
    plt.show()


#User buttons and text boxes
w_input = widgets.Text(description='w:')
b_input = widgets.Text(description='b:')
run_button_with_wb = widgets.Button(description='Run WITH w and b')
run_button_with_wb.on_click(run_lin_reg_with_wb)
run_button_without_wb = widgets.Button(description='Run WITHOUT w and b')
run_button_without_wb.on_click(run_lin_reg_without_wb)
output = widgets.Output()

display(w_input, b_input,run_button_with_wb, run_button_without_wb, output)

### Joey's Edits


In [ ]:


text_box1 = Text(value='', placeholder='x1, x2, .....', description='X')
text_box2 = Text(value='', placeholder='y1, y2, .....', description='Y')
button = Button(description='Enter')

def on_button_click(b):
    x_input = text_box1.value
    y_input = text_box2.value
    # parse the data and create a plot 
    x_values = x_input.split(',')
    y_values = y_input.split(',')
    data = [(float(x_values[i]), float(y_values[i])) for i in range(min(len(x_values), len(y_values)))]
    plt.scatter(*zip(*data))
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()


button.on_click(on_button_click)

input_box = HBox([text_box1, text_box2])
output_box = VBox([input_box, button])
output_box



In [ ]:
#Sheel's Work: plot button for w and b line

### Rupa's edits

In [ ]:
# User cell

# Getting x,y input, displaying the points
x_inp_text_box = Text(value='', placeholder='x1, x2, .....', description='X')
y_inp_text_box = Text(value='', placeholder='y1, y2, .....', description='Y')
display = Button(description='Click to display x,y on plot')

def on_display_click(b):
    x_input = x_inp_text_box.value
    y_input = y_inp_text_box.value
    # parse the data and create a plot
    x_values = x_input.split(',')
    y_values = y_input.split(',')
    data = [(float(x_values[i]), float(y_values[i]))
            for i in range(min(len(x_values), len(y_values)))]
    plt.scatter(*zip(*data))
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

display.on_click(on_display_click)
input_box_x_y = HBox([x_inp_text_box, y_inp_text_box])


# Getting w, b, displaying the points and the line given by eqn y = mx+b
w_inp_text_box = Text(value='', placeholder='slope value', description='w')
b_inp_text_box = Text(value='', placeholder='y-intercept value', description='b')
plot = Button(description='Click to plot x,y,input linear function')
def on_plot_click(b):
    x_input = x_inp_text_box.value
    y_input = y_inp_text_box.value
    w_input = int(w_inp_text_box.value)
    b_input = int(b_inp_text_box.value)
    # parse the data and create a plot of the points
    x_values = x_input.split(',')
    x_values = [int(i) for i in x_values]
    y_values = y_input.split(',')
    y_values = [int(i) for i in y_values]
    data = [(float(x_values[i]), float(y_values[i]))
            for i in range(min(len(x_values), len(y_values)))]
    plt.scatter(*zip(*data))

    # plot a line with the provided equation
    x = x_values
    y = [((w_input * int(i))+b_input) for i in x_values]
    plt.plot(x, y, '-r')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()
plot.on_click(on_plot_click)
input_box_w_b = HBox([w_inp_text_box, b_inp_text_box])

# Getting alpha, num_iters, running linear regression
alpha_inp_text_box = Text(value='', placeholder='learning rate', description='alpha')
num_iters_inp_text_box = Text(value='', placeholder='number of iterations to run linear regression for', description='Number of iterations')
run_lin_reg = Button(description='Click to run linear regression')

def run_lin_regression(b):
    x_input = x_inp_text_box.value
    y_input = y_inp_text_box.value
    alpha_input = alpha_inp_text_box.x_value
    num_iters_input = num_iters_inp_text_box.value
    # parse the data and create a plot of the points
    x_values = x_input.split(',')
    x_values = [int(i) for i in x_values]
    y_values = y_input.split(',')
    y_values = [int(i) for i in y_values]

    # PUT IN LINEAR REGRESSION FUNCTION HERE!!!
    


run_lin_reg.on_click(run_lin_regression)
input_box_alpha_num_iters = HBox([alpha_inp_text_box, num_iters_inp_text_box])



output_box = VBox([input_box_x_y, display, input_box_w_b, plot, input_box_alpha_num_iters, run_lin_reg])
output_box